In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate=0.001
total_epoch=10 #30
batch_size=128

In [3]:
# RNN은 sequential 이므로, 데이터를 몇 단계로 입력받을 지 설정해야 합니다.
# 이 부분이 좀 아쉬운데, 예시때문에 mnist를 사용하긴 했지만,
# NLP로 나중에 추가 예시를 공부하는 것이 좋을 것 같습니다.

n_input=28
n_step=28

n_hidden=128
n_class=10

In [4]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class])) # 왜 uniform으로 하지 않고 normal을 쓰는 것일까?
b = tf.Variable(tf.random_normal([n_class]))

RNN에 사용할 셀 생성

In [5]:
# 신경망 구성을 위한 셀 구성
# 다른 종류: BasicRNNCell, BasicLSTMCell, GPUCell, etc.
# LSTM: 긴 학습데이터
# GRU: Gated Recurrent Units, LSTM과 비슷하지만 구조가 좀 더 간단

cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

RNN 신경망 생성

In [6]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# 신경망 생성이 chapter 4에서 처럼 생성과정이 필요하지만, 한 줄로 간략하게 가능하다.
# RNN 신경망에 대한 공부를 하려면 검색해아 한다.
# 간단하게 설명하지면,
# states = tf.zeros(batch_size)
# for i in range(n_step):
#     outputs, states = cell(X[[:,i]], states)
# ...
#
# i에 따라서 한 단계를 학습하고 그 다음상태를 입력에 다시 학습
# 주어진 단계만큼 반복하게 상태를 전파하여 출력값을 도출해 내는 것이 RNN의 구조

RNN 출력값 -> 최종 출력값

결과를 원핫 인코딩형태로 할 것이므로 ouputs의 형태를 바꿔줘야 한다. <br>
실측값(Y) 의 형태가, [batch_size, n_class] 이르모 동일하게 바꿔준다.

In [8]:
# with tf.Session as sess:
#     sess.run(tf.global_variables_initializer())
#     print(sess.run(outputs))
# 형태를 찍어보고 싶은데.. 이렇게 하는 게 아닌가보다.. ㅠㅠㅠㅠ 

# outputs의 형태
# [batch_size, n_step, n_hidden]

outputs = tf.transpose(outputs, [1,0,2])
# [n_step, batch_size, n_hidden]

outputs = outputs[-1]
# [batch_size, n_hidden]

model = tf.matmul(outputs, W) + b

모델과 실측값 비교하여 손손실 값을 구하고<br>
신경망을 최적화

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# 이전에 배운 코드는 optimizer -> train_op로 바꿔서 했었는데.. 하나로 합치고 이름을 train_op가 아니다
# 더 이상 cost의 최적화가 목적함수가 아니라는 것 같은데, 그렇다면 목적함수는 무엇인가?
# (이전코드)
# optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
# train_op = optimizer.minimize(cost, global_step=global_step)

신경망 모델 학습

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
# batch size의 의미 학습

for epoch in range(total_epoch):
    total_cost=0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        # 입력 값(X)의 형태: [batch_size, n_step, n_input] 
        # batch_xs 의 형태도 바꿔줘야 한다.
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        
        total_cost += cost_val
    
    print('Epoch: %04d' % (epoch+1), 'Avg.cost = {:.3f}'.format(total_cost/total_batch))
    
print('최적화 완료')

정확도 확인

In [18]:
is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('accuracy: ', sess.run(accuracy*100, feed_dict={X: test_xs, Y: test_ys}))

accuracy:  97.729996


In [19]:
# 모양의 변화
# 즉, RNN 구조에 대한 행렬적 이해가 추가로 필요하다.
# CNN을 잠시 건너뛰고 왔기 때문에 그런지 몰라도 batch 에 대한 부분은 이해가 잘 가지 않는다.